In [11]:
from google.colab import drive
drive.mount('/content/drive')
import os


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
os.chdir("drive/My Drive/comp4471/yolov3/proj_code")

## Check the Directory Structure
Trained model ckpts, COCO and NYU datasets

In [0]:
!ls ../ckpt
!ls ../coco
!ls ../data

checkpoint		  cp_1.index		    cp_3.index
cp_0.data-00000-of-00001  cp_2.data-00000-of-00001  cp_4.data-00000-of-00001
cp_0.index		  cp_2.index		    cp_4.index
cp_1.data-00000-of-00001  cp_3.data-00000-of-00001
annotations  train2017	val2017
coco.names  get_coco_dataset.sh  nyu_datasets  pallete.dms


In [0]:
!ls ../ckpt

checkpoint		  cp_1.index		    cp_3.index
cp_0.data-00000-of-00001  cp_2.data-00000-of-00001  cp_4.data-00000-of-00001
cp_0.index		  cp_2.index		    cp_4.index
cp_1.data-00000-of-00001  cp_3.data-00000-of-00001


In [63]:
# Create dev set from train set (TODO: make them mutually exclusive + split out test set)
f1 = open("../nyu_train.csv", "r")
lines = f1.readlines()[:20]
f1.close()

f2 = open("../nyu_eval.csv", "w")
f2.write('\n'.join(lines) + '\n')
f2.close()

print(len(lines), "lines written")
!ls ../nyu_eval.csv

20 lines written
../nyu_eval.csv


# Train


In [0]:
!python3 main.py

# Evaluate

In [0]:
import os
import time

import tensorflow as tf
from tensorflow.keras import Input, Model

from convert_model import *
from dataloader import create_batches, load_images
from trainer import Trainer
from yolo import Yolov3_Tiny

In [76]:
# Load model

inputs = Input(shape=(416, 416, 3))
outputs, depth = Yolov3_Tiny(inputs)
model = Model(inputs, (outputs, depth))

checkpoint_path = "../ckpt/cp_4" #@param
load_status = model.load_weights(checkpoint_path)

# Sanity check
load_status.assert_consumed()  

In [0]:
 def MSE_Loss(target_y, predicted_y):
    # Use mean square error
    return tf.reduce_mean(tf.square(target_y - predicted_y))

In [0]:
model.compile(optimizer='rmsprop', loss='mean_squared_error')
# model.compile(optimizer='rmsprop', loss=MSE_Loss)

In [83]:
# Load data for evaluation
from dataloader import load_images, create_batches

eval_data_dir = "../nyu_eval.csv"
images, gts = load_images(eval_data_dir)

 30%|███       | 6/20 [00:00<00:00, 52.61it/s]

Loading traing images


100%|██████████| 20/20 [00:00<00:00, 54.39it/s]


In [84]:
# Leave some for evaluation
x_eval = images[-10:]
y_eval = gts[-10:]
x_train = images[:-10]
y_train = gts[:-10]

print(x_eval.shape, y_eval.shape)

(10, 416, 416, 3) (10, 416, 416, 1)


In [86]:
# Check model graph
%load_ext tensorboard
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.evaluate(x_eval, y_eval, steps=None, verbose=1)
          # batch_size=10, epochs=1,)
          # callbacks=[tensorboard_callback])

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ValueError: ignored

In [0]:
%tensorboard --logdir logs

In [21]:
print(images.shape)
print(gts.shape)

(1, 416, 416, 3)
(1, 416, 416, 1)


In [19]:
loss,acc = model.evaluate(images, gts, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

ValueError: ignored

## NYU Depth v2 Dataset

RMSE

# gluoncv COCO API

It does not work on colab though

In [0]:
!pip install gluoncv
!pip install mxnet

     |████████████████████████████████| 757kB 8.6MB/s 
     |████████████████████████████████| 68.7MB 49kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [0]:

from gluoncv import data, utils
from matplotlib import pyplot as plt

train_dataset = data.COCODetection('./coco',splits=['instances_train2017'])
val_dataset = data.COCODetection('./coco',splits=['instances_val2017'])
print('Num of training images:', len(train_dataset))
print('Num of validation images:', len(val_dataset))

loading annotations into memory...
Done (t=21.14s)
creating index...
index created!


OSError: ignored

In [0]:
os.chdir("./coco/train2017")

In [0]:
!ls | wc -l

ls: cannot open directory '.': Input/output error
0
